In [1]:
!pip install datasets tiktoken wandb --quiet
import tiktoken
import math
import torch
import numpy as np
import gc
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import os
import time
import pandas as pd
import re
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.0/254.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
encoder_1 = tiktoken.get_encoding("gpt2")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"shravanidhote12","key":"d238c4c92c6c26cc53fcc6382afeb017"}') # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73248 sha256=9ae17a24c81130746a078a5a3da56ff1ec29e2307c1b57d481aca75ed57e2f48
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


In [51]:
main_data = np.load('/content/drive/MyDrive/IDL/HW5/Finetune_tiral/complete_data_2.npz')
train_dataset = main_data['train']
validation_dataset   = main_data['val']
test_dataset  = main_data['test']

# Finetuning loader

In [52]:
class Fine_tuning_dataloader(torch.utils.data.DataLoader):
    def __init__(self, data, bs=32):

        self.data, self.bs, len_data = data, bs, len(data)

        temp = len_data / self.bs
        self.batch_count = math.ceil(temp)

    def __len__(self):

        return self.batch_count

    def __iter__(self):

        np.random.shuffle(self.data)

        for point in range(0, len(self.data), self.bs):

            temp = point + self.bs

            ip = self.data[point:temp, :]

            column_except_first = ip[:, 1:]

            column_of_eot_tokens = np.full((ip.shape[0], 1), encoder_1.eot_token)

            op = np.concatenate((column_except_first, column_of_eot_tokens), axis=1)

            ip_as_int64 = ip.astype(np.int64)
            op_as_int64 = op.astype(np.int64)

            yield torch.from_numpy(ip_as_int64), torch.from_numpy(op_as_int64)

val_dataset_fineloader = Fine_tuning_dataloader(validation_dataset)
test_dataset_fineloader = Fine_tuning_dataloader(test_dataset)

summary_test_data = []
qa_test_data = []
for i in tqdm(range(len(test_dataset))):
    line = encoder_1.decode(test_dataset[i]).replace("<|endoftext|>","").strip()
    if "<# generate summary#>" in line:
        summary_test_data.append(line.split("<# generate summary#>"))
    if "<#get answer#>" in line:
        qa_test_data.append(re.split(r'<#get answer#>|<#question#>', line))

100%|██████████| 68941/68941 [00:03<00:00, 19842.36it/s]


## Loading ssummary dataframe

In [53]:
number_of_data_inputs = 200
summary_task_df = pd.DataFrame(summary_test_data, columns=['article', 'highlights'])
number_of_group = summary_task_df.groupby('highlights').ngroup()
summary_task_df['GroupID'] = number_of_group + 1
summary_task_df = summary_task_df.sort_values(by='GroupID')
summary_task_df = summary_task_df[(summary_task_df['GroupID'] > number_of_data_inputs) & (summary_task_df['GroupID'] <= 2 * number_of_data_inputs)]
summary_task_df.head()

,article,highlights,GroupID
2533,"a scent could be associated with food, and the...",A Boston-area dog ate three of her owner's wri...,201
2532,"each week of dogs ingesting foreign objects, a...",A Boston-area dog ate three of her owner's wri...,201
2530,"(CNN)You might call her a watchdog, because th...",A Boston-area dog ate three of her owner's wri...,201
2531,"emergency and specialty hospital, where a vete...",A Boston-area dog ate three of her owner's wri...,201
2534,the timing of this whole incident is a little ...,A Boston-area dog ate three of her owner's wri...,201


## Loading QA dataframe

In [54]:
question_answer_df = pd.DataFrame(qa_test_data, columns=['article', 'question', "answer"])
number_of_groups_qa = question_answer_df.groupby('question').ngroup()
question_answer_df['GroupID'] = number_of_groups_qa + 1
question_answer_df = question_answer_df.sort_values(by='GroupID')
question_answer_df = question_answer_df[(question_answer_df['GroupID'] > number_of_data_inputs) & (question_answer_df['GroupID'] <= 2 * number_of_data_inputs)]
question_answer_df.head()

,article,question,answer,GroupID
2549,Trade liberalization may shift economic inequa...,"Compared to other causes, the effect of trade ...",minor,201
2550,that low-skilled jobs have been replaced by ma...,"Compared to other causes, the effect of trade ...",minor,201
2377,Jacksonville is the most populous city in Flor...,"Compared to the rest of Florida, how does Jack...",largest,202
1691,"In humans, this response is activated by compl...",Complement proteins bind to what kind of molec...,carbohydrates,203
4422,"In particular, this norm gets smaller when a n...",Completing Q with respect to what will produce...,the absolute value,204


# Model classes

In [55]:
# # Architecture Configuration

n_head = 6
num_heads = 6
n_embd = 512
n_layer = 6
dropout = 0.2
block_size = 256
batch_size = 40
learning_rate = 3e-4

vocab_size = encoder_1.n_vocab

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        # Linear layers for key, query, and value
        self.linear_layers = nn.ModuleList([
            nn.Linear(n_embd, head_size, bias=False) for _ in range(3)
        ])
        self.key_layer, self.query_layer, self.value_layer = self.linear_layers
        # Lower triangular mask for attention
        identity_matrix = torch.eye(block_size)
        lower_triangular_mask = torch.tril(identity_matrix)
        self.register_buffer('tril', lower_triangular_mask)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        # Self-attention mechanism
        batch_size = x.shape[0]
        time_steps = x.shape[1]
        channels = x.shape[2]
        key, query = self.key_layer(x), self.query_layer(x)
        key_transposed = key.transpose(-2, -1)
        attention_scores = torch.matmul(query, key_transposed) / torch.sqrt(torch.tensor(key.shape[-1], dtype=torch.float32))
        attention_mask = torch.where(self.tril[:time_steps, :time_steps] == 0, float('-inf'), attention_scores)
        attention_probs = self.drop(torch.nn.functional.log_softmax(attention_mask, dim=-1).exp())
        value = self.value_layer(x)
        output = attention_probs @ value
        return output

# Define Multi-Head Attention module
class MultiHeadAttention(nn.Module):
    """ Multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList()
        for _ in range(num_heads):
            self.heads.append(Head(head_size))
        self.projection = nn.Linear(head_size * num_heads, n_embd)
        self.drop = nn.Dropout(dropout)

    def forward(self, x):
        # Combine multiple self-attention heads
        individual_heads = [head_module(x) for head_module in self.heads]
        concatenated_heads_expanded = torch.cat(individual_heads, dim=-1)
        projected_out = self.projection(concatenated_heads_expanded)
        dropped_out = self.drop(projected_out)
        return dropped_out

# Define FeedForward module
class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout):
        super(FeedForward, self).__init__()
        self.layer1 = nn.Linear(n_embd, 4 * n_embd)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(4 * n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # FeedForward neural network
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.dropout(x)
        return x

# Define a Block in the GPT model
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        self.self_attention = MultiHeadAttention(n_head, (n_embd // n_head))
        self.feed_forward = FeedForward(n_embd, dropout)
        self.linearlayer1, self.linearlayer2 = nn.LayerNorm(n_embd), nn.LayerNorm(n_embd)

    def forward(self, x):
        # Block in the GPT model
        x = x + self.self_attention(self.linearlayer1(x)) + self.feed_forward(self.linearlayer2(x))
        return x

# Define the GPT Language Model
class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb_tbl = nn.Embedding(vocab_size, n_embd)
        self.pos_emb_tbl = nn.Embedding(block_size, n_embd)
        attention_blocks = [Block(n_embd, n_head=n_head) for _ in range(n_layer)]
        self.attention_blocks = nn.Sequential(*attention_blocks)
        self.layer_norm_final = nn.LayerNorm(n_embd)
        self.mode_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        # Initialize weights for linear and embedding layers
        if isinstance(module, (nn.Linear, nn.Embedding)):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            torch.nn.init.zeros_(module.bias) if hasattr(module, 'bias') and module.bias is not None else None

    def forward(self, idx, targets=None):
        # Forward pass through the GPT model
        embedding_indices = idx
        token_embeddings = self.token_emb_tbl(embedding_indices)
        pos_indices = torch.arange(idx.shape[1], device=device)
        positional_embeddings = self.pos_emb_tbl(pos_indices).unsqueeze(0)
        normalized_embeddings = self.layer_norm_final(self.attention_blocks(token_embeddings + positional_embeddings.expand_as(token_embeddings)))
        logits = self.mode_head(normalized_embeddings)
        loss = F.cross_entropy(logits.flatten(0, 1), targets.flatten()) if targets is not None else None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # Generate new tokens
        for _ in range(max_new_tokens):
            currentlogits, _ = self.forward(idx[:, -block_size:])
            currentlogits = currentlogits[:, -1, :]
            nexttoken = torch.multinomial(F.softmax(currentlogits, dim=-1), num_samples=1).view(-1, 1)
            idx = torch.hstack((idx, nexttoken))
        return idx


In [57]:
def generate_tokens_greedily(initial_indices, newtokens):
  for i in range(newtokens):
        indices_cond = initial_indices[:, -block_size:]
        with torch.no_grad():
          output_logits, loss = model(indices_cond)
        output_logits = output_logits[:, -1, :]
        prob = torch.nn.functional.softmax(output_logits, dim=-1)
        max_indices = torch.max(prob, dim=-1, keepdim=True)[1]
        initial_indices = torch.cat((initial_indices, max_indices), dim=1)
  return initial_indices

# Wandb Run

In [46]:
# wandb.finish()
import wandb
wandb.login(key="bc23ae38b83159616a86ecc84dc25bc2e82d4da4")

run = wandb.init(
    name = "hw5_finetuning-Kaggle-2", ## Wandb creates random run names if you skip this field
#     reinit = True,  ### Allows reinitalizing runs when you re-run this cell
    id = "ii86awr7",  # Insert specific run id here if you want to resume a previous run
    resume = "must",  ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw5-ablations")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train_loss,2.76955
valid_loss,3.24345


# Initialize model

In [58]:
model = GPTLanguageModel()
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, factor =0.1)

In [59]:
url = wandb.restore("finet2_0_20000_2.8563.pth").name
model.load_state_dict(torch.load(url,map_location=torch.device(device))["model_state_dict"])
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(torch.load(url)["optimizer_state_dict"])

## Test the model

### Summary task

In [60]:
eval_model = model.eval()

def test_summary(row):
  article = row["article"]
  temp = article+"<# generate summary#>"
  article = torch.tensor(encoder_1.encode(temp))
  article = torch.unsqueeze(article,0)
  index     = generate_tokens_greedily(article.to(device), block_size-article.shape[0])
  complete_sentence = encoder_1.decode(index.detach().tolist()[0])
  input, output = complete_sentence.split("<# generate summary#>")
  return output

len_summary_task = len(summary_task_df)
for i, r in tqdm(summary_task_df.iterrows(), total=len_summary_task):
    summary_task_df.at[i, 'prediction'] = test_summary(r)

summary_task_df.to_csv("/content/drive/MyDrive/IDL/HW5/Final_code_files/Results/test_summary.csv",index=False)
summary_task_df.head()

100%|██████████| 1018/1018 [1:01:45<00:00,  3.64s/it]


,article,highlights,GroupID,prediction
2533,"a scent could be associated with food, and the...",A Boston-area dog ate three of her owner's wri...,201,Mocha's owners are concerned about the dog's w...
2532,"each week of dogs ingesting foreign objects, a...",A Boston-area dog ate three of her owner's wri...,201,The dog was found dead in a field in the Newfo...
2530,"(CNN)You might call her a watchdog, because th...",A Boston-area dog ate three of her owner's wri...,201,"Doberman's wife, who is the most polite person..."
2531,"emergency and specialty hospital, where a vete...",A Boston-area dog ate three of her owner's wri...,201,"The woman who died in a car accident in Mocha,..."
2534,the timing of this whole incident is a little ...,A Boston-area dog ate three of her owner's wri...,201,Apple's iPhone 5S has been in the works for th...


In [ ]:
def test_summary_2(row):
  article = row["prediction"].replace(" ## "," .")
  article = torch.tensor(encoder_1.encode(article+"<# generate summary#>"))
  article = torch.unsqueeze(article,0)
  index     = generate_tokens_greedily(article.to(device), block_size-article.shape[0])
  complete_sentence = encoder_1.decode(index.detach().tolist()[0])
  input, output = complete_sentence.split("<# generate summary#>")
  return output

cumulative_sum_df = summary_task_df.groupby('GroupID').agg({
    'article':    ' -- '.join,
    'prediction': ' ## '.join,
    'highlights': lambda p: p.iloc[0]
}).reset_index()

for i, r in tqdm(cumulative_sum_df.iterrows(), total=len(cumulative_sum_df)):
    cumulative_sum_df.at[i, 'prediction'] = test_summary_2(r)

cumulative_sum_df.to_csv("/content/drive/MyDrive/IDL/HW5/Final_code_files/Results/final_test_summary.csv",index=False)
cumulative_sum_df.head()

### Question-answer task

In [ ]:
def test_question_answer(row):
  article = row["article"]+"<#question#>"+row["question"]
  temp = article+"<#get answer#>"
  article = torch.tensor(encoder_1.encode(temp))
  article = torch.unsqueeze(article,0)
  index = generate_tokens_greedily(article.to(device), block_size-article.shape[0])
  detached_list = index.detach().tolist()[0]
  complete_sentence = encoder_1.decode(detached_list)
  input, output  = complete_sentence.split("<#get answer#>")
  return output

len_qa_task = len(question_answer_df)
for i, r in tqdm(question_answer_df.iterrows(), total=len_qa_task):
    question_answer_df.at[i, 'prediction'] = test_question_answer(r)

question_answer_df = question_answer_df.drop_duplicates(subset='GroupID', keep=False)
question_answer_df.to_csv("/content/drive/MyDrive/IDL/HW5/Final_code_files/Results/test_question_answer.csv",index=False)
question_answer_df.head()

# ROGUE and BLEU Scoring

In [ ]:
!pip install nltk
!pip install rouge-score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

In [ ]:
def obtain_bleu(prediction, target_output):
    bleu_score = sentence_bleu([target_output.split()], prediction.split())
    return bleu_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def obtain_rouge(prediction, target_output):
    rogue_scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
    scores = rogue_scorer.score(target_output, prediction)
    return scores['rouge1'].fmeasure

In [ ]:
def obtain_score(df):
    df['bleu_score'] = df.apply(lambda row: obtain_bleu(row['final_prediction'], row['highlights']), axis=1)
    df['rouge_scores'] = df.apply(lambda row: obtain_rouge(row['final_prediction'], row['highlights']), axis=1)
    return df

### Summary task

In [ ]:
cumulative_sum_df_2 = obtain_score(cumulative_sum_df)
average_bleu = cumulative_sum_df_2['bleu_score'].mean()
average_rouge = cumulative_sum_df_2['rouge_scores'].mean()
print("\nThe Average BLEU Score for Summary task:", average_bleu)
print("\nThe Average ROUGE Scores for Summary task:", average_rouge)

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
cumulative_sum_df_2.head(10)

In [ ]:
cumulative_sum_df_2.to_csv("/content/drive/MyDrive/IDL/HW5/Final_code_files/Results/scored_summary.csv", index = False)

### Question-Answer task

In [ ]:
question_answer_df = obtain_score(question_answer_df)
average_bleu = question_answer_df['bleu_score'].mean()
average_rouge = question_answer_df['rouge_scores'].mean()
print("\nThe Average BLEU Score for question-answer task:", average_bleu)
print("\nThe Average ROUGE Score for question-answer task:", average_rouge)
question_answer_df.head(5)

In [ ]:
question_answer_df.to_csv("/content/drive/MyDrive/IDL/HW5/Final_code_files/Results/scored_QA.csv", index = False)